In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 5s (55.5 kB/s)
Reading package li

In [2]:
# Connect to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-08-19 14:49:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.01MB/s    in 0.2s    

2022-08-19 14:49:31 (5.01 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

## ***Extract the data***

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://my-data-class-burketi.s3.amazonaws.com/amazon_reviews_us_Automotive_v1_00.tsv" 
spark.sparkContext.addFile(url)
automotive_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv"), sep='\t', header=True, inferSchema=True, timestampFormat="yyyy/MM/dd")

automotive_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

# Transform the data

In [6]:
#number of rows in this dataset
f"The number of rows in this dataset is {automotive_reviews_df.count()}"

'The number of rows in this dataset is 3514942'

In [7]:
f"the number of columns in this dataset are {len(automotive_reviews_df.columns)}"

'the number of columns in this dataset are 15'

In [8]:
f"the number of columns are {automotive_reviews_df.columns}"

"the number of columns are ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']"

## Transform the data to fit into the schema

A postgres sql database has been created. the database has 4 tables. In this section 4 dataframes are created. This dataframes will be loaded to the sql database. Therefore the datafrom spark is transformed to ensure that it is compatible with tables in the postgres database

In [9]:
#checking the column datatypes
automotive_reviews_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
#Changing the column datatypes as some are not true and forming a new dataframe

from pyspark.sql.types import * 
automotive_reviews = automotive_reviews_df.withColumn("customer_id", automotive_reviews_df["customer_id"].cast(IntegerType()))\
                          .withColumn("product_parent", automotive_reviews_df["product_parent"].cast(IntegerType()))\
                          .withColumn("star_rating", automotive_reviews_df["star_rating"].cast(IntegerType()))\
                          .withColumn("helpful_votes", automotive_reviews_df["helpful_votes"].cast(IntegerType()))\
                          .withColumn("total_votes", automotive_reviews_df["total_votes"].cast(IntegerType()))\
                          .withColumn("review_date", automotive_reviews_df["review_date"].cast(DateType()))

In [11]:
#checking the changes
automotive_reviews.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [13]:
#Now creating tables Create review_id_df that will be sent to review_id_table
automotive_id_df = automotive_reviews.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
automotive_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

In [14]:
#Now creating tables Create review_id_df that will be sent to products_id_table
products_id_df = automotive_reviews.select([ "product_id", "product_title"])
products_id_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00LPRXQ4Y|17" 2003-2006 For...|
|B000C7S0TO|Spectra Premium C...|
|B000CO9WE4|K&N E-4665 High P...|
|B000GKD5NI|Suncutters Rear W...|
|B009SDA7TE|Lug Nuts Landcrui...|
|B00KV15KRG|Fits 2007-2009 Ho...|
|B0002JMAKW|Castrol 12614 Dot...|
|B00XJKMM6S|New Power Window ...|
|B000C5CEKC|Motorad MGC-791 L...|
|B00GD9R2A8|Genuine Ford AL3Z...|
|B00RM327NG|Air Vent Smartpho...|
|B0071NL7UQ|FDJ New Replaceme...|
|B00DSOL0FY|Omix-Ada 17402.09...|
|B00GRV48TK|GMB 530-2480 Fuel...|
|B000E4PCGE|00-03 NISSAN SENT...|
|B005Z8HAUK|Slime 40026 2X He...|
|B0043AV4NK|MegaBrand N1 4" B...|
|B00CDHD76C|Jackly 45-in-One ...|
|B000CF1XO0|Bosch 3397118942 ...|
|B003VLBPJA|Heavy-Duty 15" Ca...|
+----------+--------------------+
only showing top 20 rows



In [15]:
#dropping duplicates from product_id_df
products_id_df = products_id_df.dropDuplicates(["product_id"])

In [16]:
products_id_df.count()

762982

In [18]:
#are there duplicates in the data?
if products_id_df.count() > products_id_df.dropDuplicates(["product_id"]).count():
    raise ValueError('Data has duplicates')

In [19]:
products_id_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [20]:
# Create customers table 
customers_df = automotive_reviews.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28258386|             3|
|   18634862|             3|
|    2072312|             3|
|   33132797|            15|
|   14539137|             1|
|   14552054|             5|
|    9347763|             1|
|   43626894|             2|
|   27314089|            47|
|   36829268|             1|
|   24540309|             2|
|    8501205|             1|
|    4361670|             2|
|   30416085|             2|
|    8306731|             1|
|   11491871|             1|
|   46406103|             1|
|   18417791|             1|
|   51299913|             1|
|   20587872|             4|
+-----------+--------------+
only showing top 20 rows



In [21]:
# Check the customers table schema 
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [22]:
# Change customer_count to integer type
customers_df =customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.printSchema()


root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [24]:
vine_df = automotive_reviews.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RAB23OVFNCXZQ|          1|            0|          0|   N|
|R3NORADVJO6IE6|          5|            0|          0|   N|
|R299F4SO98S5OO|          5|            1|          1|   N|
|R2DA9DOT03UW6I|          5|            2|          3|   N|
|R2OGCH681EQHU6|          5|            0|          0|   N|
| R2JMKIC16MHD7|          5|            2|          2|   N|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|
|R1N8XWFDK4QACP|          5|            0|          0|   N|
|R19VFLX6MRU0CX|          5|            0|          0|   N|
| RLTD8YDGTP9PT|          1|            0|          0|   N|
| R78B5LCFJ52SR|          4|            0|          0|   N|
|R33SKWL0HEQIQ9|          5|            0|          0|   N|
|R1HIOV0UX7P13D|          5|            0|          0|   N|
| R4WS0E0MSP9DH|          5|            

In [25]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# Load data to Posgress Database

In [26]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [27]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-bootcamp.cgidpfd2p1nl.us-east-1.rds.amazonaws.com:5432/amazon_automotive_reviews_db"
config = {"user":"root", 
          "password": "bootcamp", 
          "driver":"org.postgresql.Driver"}


In [28]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)


In [30]:
# Write DataFrame to automotive_id_table table in RDS

automotive_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)


In [31]:
# Write DataFrame to vine_table table in RDS

vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)

In [32]:
# Write DataFrame to products table in RDS

products_id_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)
